# Next.js Data Fetching: SSR, SSG, and ISR

## Introduction
Next.js supports three primary data fetching strategies in its App Router: **Server-Side Rendering (SSR)**, **Static Site Generation (SSG)**, and **Incremental Static Regeneration (ISR)**. These methods determine when and how pages render and fetch data, balancing performance, freshness, and cost. By default, components are server components unless marked with `'use client'`, enabling server-side execution for data fetching.

## Server-Side Rendering (SSR)
SSR renders pages on the server for each request, ideal for dynamic, user-specific content like personalized dashboards.

- Default behavior: Components without `'use client'` run on the server; `console.log` statements appear in server logs, not browser console.[1]
- Data fetching occurs server-side via `fetch` or database calls during each request.
- **Dynamic Functions** trigger SSR: `cookies()`, `headers()`, or `searchParams` force request-time rendering.
- To force SSR explicitly: Export `export const dynamic = 'force-dynamic'` in the page file, preventing static pre-rendering.
- Trade-offs: Higher server load but always fresh data; no caching by default unless specified.

Example: Adding `dynamic: 'force-dynamic'` changes build output from "Static" to "Dynamic" page.

## Static Site Generation (SSG)
SSG pre-renders pages at **build time** (`npm run build`), generating static HTML for fast CDN delivery without network calls per request.

- Triggers automatically: Pages without dynamic functions or uncached fetches become static.[3]
- No network calls during rendering means instant loads; cached via Data Cache.
- Build log shows "○ (Static)" for qualifying pages like simple homepages.[1]
- Benefits: Optimal for unchanging content (e.g., marketing pages, blog posts); reduces costs and latency.[6]

Example: A basic page with static text renders as "pre-rendered as static content" during build.[1]

## Incremental Static Regeneration (ISR)
ISR combines SSG speed with periodic updates, regenerating static pages in the background after a set interval.

- Uses Next.js `fetch` with `{ next: { revalidate: seconds } }` (e.g., `revalidate: 3600` for hourly updates).[7][1]
- Serves cached static page initially; regenerates on next request after interval for freshness.[2]
- **Caching Control**:
  | Option | Behavior |
  |--------|----------|
  | Default `fetch` | Caches response; deduplicates same-URL calls [8] |
  | `{ cache: 'no-store' }` | Disables cache; fetches fresh data every time (SSR-like) [1] |
  | `{ tags: ['collection'] }` | Enables on-demand revalidation via `revalidateTag` [5] |

Example: Fetching posts from JSONPlaceholder API caches first load; `{ revalidate: 3600 }` updates hourly.[1]

## Next.js Fetch API Specifics
Next.js extends native `fetch` for optimization:
- **Auto-caching**: GET requests cache by default; shared across users.[8]
- **Exceptions**: No caching in Server Actions or Route Handlers (POST).[1]
- Parallel fetching: Use `Promise.all` for independent requests to avoid sequential blocking.[8]
- Deduplication: Same URL/options in one render combine into one request.[8]

**Code Example** (ISR fetch):
```jsx
const data = await fetch('https://api.example.com/posts', {
  next: { revalidate: 3600 } // Revalidate every hour
});
```

## Exceptions and Best Practices
- **Static by Default**: Next.js prefers SSG; opts into SSR only if dynamic APIs detected.[9][3]
- **Streaming**: Use `loading.js` or `<Suspense>` for non-blocking UIs during fetches.[8]
- Rule of Thumb: Use SSG/ISR for infrequent changes; SSR for real-time needs. Check docs for edge cases.[5][1]

## Summary of Main Takeaways
- **SSR**: Per-request rendering for dynamic data (`dynamic = 'force-dynamic'`).
- **SSG**: Build-time static pages (default for static content).
- **ISR**: Cached static with timed revalidation (`{ next: { revalidate: N } }`).
Master via Next.js docs; 95% covered here—test in projects for mastery.